In [2]:
%reset -f
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn import cross_validation
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.ensemble import GradientBoostingRegressor
from math import sqrt


In [5]:
rentdata = pd.read_csv('version2.csv')
y=rentdata['price'].values
rentdata=rentdata.drop('price',axis=1)

We define the feature we want to predict (price), ant split our training and testing sets.

In [6]:
X=rentdata.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,random_state=44)



In [7]:
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test= scaler.transform(X_test)

In [8]:
print X_train.shape
print X_test.shape
print X.shape


(4556L, 10L)
(507L, 10L)
(5063L, 10L)


In [9]:
#Creates a vector (x,x^2,x^3,....,x^i)
def extend(x,n):
    x_extend=x
    for i in range(1,n): x_extend=np.c_[x_extend,x**(i+1)]
    return x_extend

In [10]:
#Root mean squared error
def rmse(y,yhat):
    dif=y-yhat
    N=len(y)
    return sqrt((1./N)*np.dot(dif.T,dif))



In [11]:
CMAX=20
Err=np.zeros((CMAX,3,2))
reg_meth=['Linear Regression','Ridge regressor','Gradient Boosting Regressor']


In [12]:
#Linear Regressor
lreg = LinearRegression(fit_intercept=True,normalize=False)
for i in range(CMAX):
    lreg.fit(extend(X_train,i+1),y_train)
    yhat_in=lreg.predict(extend(X_train,i+1))
    Err[i,0,0]=rmse(y_train,yhat_in)
    yhat_out=lreg.predict(extend(X_test,i+1))
    Err[i,0,1]=rmse(y_test,yhat_out)

In [13]:
#Ridge Regressor
rid = Ridge()
for i in range(CMAX):
    rid.fit(extend(X_train,i+1), y_train)
    yhat_in=rid.predict(extend(X_train,i+1))
    Err[i,1,0]=rmse(y_train,yhat_in)
    yhat_out=rid.predict(extend(X_test,i+1))
    Err[i,1,1]=rmse(y_test,yhat_out)

In [14]:
#Gradient Boosting Regressor
gbr=GradientBoostingRegressor()
for i in range(CMAX):
    gbr.fit(extend(X_train,i+1), y_train)
    yhat_in=gbr.predict(extend(X_train,i+1))
    Err[i,2,0]=rmse(y_train,yhat_in)
    yhat_out=gbr.predict(extend(X_test,i+1))
    Err[i,2,1]=rmse(y_test,yhat_out)
    
    

In [16]:
Ein_min=Err[:,:,0].min()
Eout_min=Err[:,:,1].min()
idx_inmin=np.asarray([np.where(Err[:,:,0]==Ein_min)[0][0],np.where(Err[:,:,0]==Ein_min)[1][0]])
idx_outmin=np.asarray([np.where(Err[:,:,1]==Eout_min)[0][0],np.where(Err[:,:,1]==Eout_min)[1][0]])
print Ein_min,Eout_min
print idx_inmin
print idx_outmin

 98.6486884813 105.634328219
[0 2]
[0 2]


In [17]:
print '     \t Linear Regressor \t         Ridge Regressor \t  Gradient Boosting Regressor'
print 'C    \t  Ein         Eout       \t  Ein           Eout   \t          Ein          Eout'
for i in range(CMAX):
    print  i+1,'  ',Err[i,0,0],' ',Err[i,0,1],' ',Err[i,1,0],' ',Err[i,1,1],' ',Err[i,2,0],' ',Err[i,2,1]
print '\n\nThe minimum Ein is',Ein_min,'for',reg_meth[idx_inmin[1]],',complexity',idx_inmin[0]+1
print '\n\nThe minimum Eout is',Eout_min,'for',reg_meth[idx_outmin[1]],',complexity',idx_outmin[0]+1  

     	 Linear Regressor 	         Ridge Regressor 	  Gradient Boosting Regressor
C    	  Ein         Eout       	  Ein           Eout   	          Ein          Eout
1    568.916323952   567.407849445   568.916414517   567.389616799   98.6486884813   105.634328219
2    425.977908286   420.218280316   425.979276046   420.066813244   100.063896487   108.402748709
3    360.70088846   350.916178347   360.704833338   350.839263686   100.063896487   109.82077164
4    329.989766793   339.85298328   329.993895263   339.581907107   100.063896487   109.309703189
5    316.723963331   723.893010014   316.730738711   725.391895793   100.063896487   110.088709574
6    311.863561288   468.458005844   311.870519752   463.933663467   100.063896487   109.227941209
7    305.798835924   1563.73731854   305.806135144   1591.32720929   100.063896487   111.359591184
8    298.352976307   13733.3022337   298.359426651   13551.0575175   100.063896487   109.471396064
9    295.973744482   24829.3947007   295.98198

In [18]:
X_scaled=scaler.fit_transform(X)
if idx_outmin[1]==0: freg = LinearRegression()
if idx_outmin[1]==1: freg = Ridge() 
if idx_outmin[1]==2: freg = GradientBoostingRegressor()
i=idx_outmin[0]
freg.fit(extend(X_scaled,i+1),y)
yhat_in=freg.predict(extend(X_scaled,i+1))
Ein=rmse(y,yhat_in)
#yhat_out=freg.predict()

print 'Ein=',Ein#,'\t Eout=',Eout

Ein= 97.5461841012
